In [1]:
#decoding the data
import numpy as np
import tensorflow as tf
import helpers

import os



SEPCTURAL_SAMPLES = 40

#FEATURE_DIM = SEPCTURAL_SAMPLES*6*2   # This data set = 30
FEATURE_DIM=30



#INTER_DIM = 120  #60
INTER_DIM=121


#OUT_DIM = 6#len(idDict)  #66
OUT_DIM = 66


#WIDE = 20  #time step  #40
WIDE=121





BATCH_SIZE = 16
TOTAL_ITER_NUM = 1000000000

In [2]:
import pdb
def read_audio_csv(filename_queue):
	reader = tf.TextLineReader()
	key, value = reader.read(filename_queue)

	defaultVal = [[0.] for idx in range(WIDE*FEATURE_DIM + OUT_DIM)]

	fileData = tf.decode_csv(value, record_defaults=defaultVal)

	features = fileData[:WIDE*FEATURE_DIM]
  
	features = tf.reshape(features, [WIDE, FEATURE_DIM])
	labels = fileData[WIDE*FEATURE_DIM:] 

	return features, labels

def input_pipeline(filenames, batch_size, shuffle_sample=True, num_epochs=None):
	filename_queue = tf.train.string_input_producer(filenames, shuffle=shuffle_sample)

	# filename_queue = tf.train.string_input_producer(filenames, num_epochs=TOTAL_ITER_NUM*EVAL_ITER_NUM*10000000, shuffle=shuffle_sample)
	example, label = read_audio_csv(filename_queue)
	min_after_dequeue = 2000#int(0.4*len(csvFileList)) #1000
	capacity = min_after_dequeue + 3 * batch_size
	if shuffle_sample:
		example_batch, label_batch = tf.train.shuffle_batch(
			[example, label], batch_size=batch_size, num_threads=16, capacity=capacity,
			min_after_dequeue=min_after_dequeue)
	else:
		example_batch, label_batch = tf.train.batch(
			[example, label], batch_size=batch_size, num_threads=16)
	return example_batch, label_batch

In [3]:
csvFileList = []

csvDataFolder1 = os.path.join('N_Z')
#csvDataFolder1 = os.path.join('Data_V1')

print(csvDataFolder1)
orgCsvFileList = os.listdir(csvDataFolder1)

len(orgCsvFileList)


N_Z


195

In [4]:
for csvFile in orgCsvFileList:
	if csvFile.endswith('.csv'):
		csvFileList.append(os.path.join(csvDataFolder1, csvFile))

In [5]:
len(csvFileList)

195

In [6]:
batch_size=16
batch_feature, batch_label = input_pipeline(csvFileList, batch_size)
#print(batch_feature)
#pdb.set_trace() 
#print(batch_label)

In [7]:
sensor_inputs = tf.expand_dims(batch_feature, axis=3)

In [8]:
sensor_inputs

<tf.Tensor 'ExpandDims:0' shape=(16, 121, 30, 1) dtype=float32>

In [9]:
output_keep_prob=0.5
train=True

gru_cell1 = tf.contrib.rnn.GRUCell(128)
if train:
    gru_cell1 = tf.contrib.rnn.DropoutWrapper(gru_cell1, output_keep_prob=0.5)

gru_cell2 = tf.contrib.rnn.GRUCell(128)
if train:
    gru_cell2 = tf.contrib.rnn.DropoutWrapper(gru_cell2, output_keep_prob=0.5)

In [10]:
used = tf.sign(tf.reduce_max(tf.abs(batch_feature), reduction_indices=2))

In [11]:
#print(batch_feature)

In [12]:
length = tf.reduce_sum(used, reduction_indices=1) #(BATCH_SIZE)

In [13]:
length

<tf.Tensor 'Sum:0' shape=(16,) dtype=float32>

In [14]:
length = tf.cast(length, tf.int64)
length

<tf.Tensor 'Cast:0' shape=(16,) dtype=int64>

In [15]:
sensor_inputs

<tf.Tensor 'ExpandDims:0' shape=(16, 121, 30, 1) dtype=float32>

In [16]:
#cell = tf.contrib.rnn.MultiRNNCell([gru_cell1, gru_cell2])
cell = tf.contrib.rnn.MultiRNNCell([gru_cell1,gru_cell2])

In [17]:
init_state = cell.zero_state(BATCH_SIZE, tf.float32)

In [18]:
cell_output, final_stateTuple = tf.nn.dynamic_rnn(cell, batch_feature, sequence_length=length, initial_state=init_state, time_major=False)

In [19]:
cell_output

<tf.Tensor 'rnn/transpose_1:0' shape=(16, 121, 128) dtype=float32>

In [20]:
inputs=batch_feature
mask = tf.sign(tf.reduce_max(tf.abs(inputs), reduction_indices=2, keepdims=True))
print(mask)
mask = tf.tile(mask, [1,1,128]) # (BATCH_SIZE, WIDE, INTER_DIM)
avgNum = tf.reduce_sum(mask, reduction_indices=1) #(BATCH_SIZE, INTER_DIM)
print mask

Tensor("Sign_1:0", shape=(16, 121, 1), dtype=float32)
Tensor("Tile:0", shape=(16, 121, 128), dtype=float32)


In [21]:
print(mask)
print(avgNum )

Tensor("Tile:0", shape=(16, 121, 128), dtype=float32)
Tensor("Sum_1:0", shape=(16, 128), dtype=float32)


In [22]:
layers = tf.contrib.layers 
sum_cell_out = tf.reduce_sum(cell_output*mask, axis=1, keep_dims=False)
avg_cell_out = sum_cell_out/avgNum

logits = layers.fully_connected(avg_cell_out, OUT_DIM, activation_fn=None, scope='output')


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [23]:
predict = tf.argmax(logits, axis=1)
predict
batchLoss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=batch_label)
loss = tf.reduce_mean(batchLoss)
t_vars = tf.trainable_variables()

regularizers = 0.
for var in t_vars:
	regularizers += tf.nn.l2_loss(var)
loss += 5e-4 * regularizers


discOptimizer = tf.train.AdamOptimizer(
		learning_rate=1e-4, 
		beta1=0.5,
		beta2=0.9
	).minimize(loss, var_list=t_vars)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
tf.initialize_all_variables()

saver = tf.train.Saver()

lossz=[]
accz=[]
epochz=[]

with tf.Session() as sess:
	tf.train.start_queue_runners()
	sess.run(tf.local_variables_initializer())
	tf.global_variables_initializer().run()

	coord = tf.train.Coordinator()
	threads = tf.train.start_queue_runners(coord=coord)

	for iteration in xrange(1000000000):
		print("Iteration NUmber :", iteration)        
		_, lossV, _trainY, _predict = sess.run([discOptimizer, loss, batch_label, predict])
     
		print(lossV)       
		_label = np.argmax(_trainY, axis=1)
		_accuracy = np.mean(_label == _predict)   
		coord.request_stop()
		coord.join(threads)
 

		if iteration % 200 == 0:
			lossz.append(lossV) 
			epochz.append(iteration)
			accz.append(_accuracy)
            
        
		if iteration % 20000 == 0:
			save_path = saver.save(sess, "/home/dl/Documents/Shamane/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check_n",global_step=iteration)
			print("Model saved in path: %s" % save_path)
            


Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Iteration NUmber :', 0)
4.6244864
Model saved in path: /home/dl/Documents/Shamane/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check_n-0
('Iteration NUmber :', 1)
4.761215
('Iteration NUmber :', 2)
4.4360027
('Iteration NUmber :', 3)
4.52403
('Iteration NUmber :', 4)
4.4792213
('Iteration NUmber :', 5)
4.473071
('Iteration NUmber :', 6)
4.51535
('Iteration NUmber :', 7)
4.571396
('Iteration NUmber :', 8)
4.285109
('Iteration NUmber :', 9)
4.427572
('Iteration NUmber :', 10)
4.2715144
('Iteration NUmber :', 11)
4.4528313
('Iteration NUmber :', 12)
4.2678022
('Iteration NUmber :', 13)
4.7397027
('Iteration NUmber :', 14)
4.5080447
('Iteration NUmber :', 15)
4.407243
('Iteration NUmber :', 16)
4.137822
('Iteration NUmber :', 17)
4.302174
('Iteration NUmber :', 18)
4.266453
('Iteration NUmber :', 19)
4.394599
('Iteration NUmber :', 20)
4.3461747
('Iteration NUmber :', 21)
4.049313
('Iteration NUmber 

2.3870883
('Iteration NUmber :', 210)
2.5823917
('Iteration NUmber :', 211)
3.2650764
('Iteration NUmber :', 212)
2.6477635
('Iteration NUmber :', 213)
2.6321845
('Iteration NUmber :', 214)
2.5205634
('Iteration NUmber :', 215)
2.4460268
('Iteration NUmber :', 216)
2.3678622
('Iteration NUmber :', 217)
2.6571035
('Iteration NUmber :', 218)
3.129098
('Iteration NUmber :', 219)
2.718996
('Iteration NUmber :', 220)
2.9981391
('Iteration NUmber :', 221)
3.243155
('Iteration NUmber :', 222)
2.8836129
('Iteration NUmber :', 223)
2.3375983
('Iteration NUmber :', 224)
2.576523
('Iteration NUmber :', 225)
2.6783533
('Iteration NUmber :', 226)
2.807599
('Iteration NUmber :', 227)
2.6692877
('Iteration NUmber :', 228)
2.3703969
('Iteration NUmber :', 229)
2.543626
('Iteration NUmber :', 230)
2.438503
('Iteration NUmber :', 231)
2.5206513
('Iteration NUmber :', 232)
2.6002378
('Iteration NUmber :', 233)
2.0022433
('Iteration NUmber :', 234)
2.4487493
('Iteration NUmber :', 235)
2.7400208
('Iterati

2.216537
('Iteration NUmber :', 427)
1.5354377
('Iteration NUmber :', 428)
1.5623713
('Iteration NUmber :', 429)
1.588553
('Iteration NUmber :', 430)
1.4112017
('Iteration NUmber :', 431)
2.2484856
('Iteration NUmber :', 432)
1.8697119
('Iteration NUmber :', 433)
2.8930516
('Iteration NUmber :', 434)
2.305491
('Iteration NUmber :', 435)
2.0801892
('Iteration NUmber :', 436)
2.3314605
('Iteration NUmber :', 437)
2.1734557
('Iteration NUmber :', 438)
2.187449
('Iteration NUmber :', 439)
2.0532036
('Iteration NUmber :', 440)
2.311747
('Iteration NUmber :', 441)
2.5754354
('Iteration NUmber :', 442)
1.8797445
('Iteration NUmber :', 443)
2.1176484
('Iteration NUmber :', 444)
1.713107
('Iteration NUmber :', 445)
1.2473779
('Iteration NUmber :', 446)
2.012706
('Iteration NUmber :', 447)
1.807887
('Iteration NUmber :', 448)
2.0845816
('Iteration NUmber :', 449)
2.1268327
('Iteration NUmber :', 450)
1.6192555
('Iteration NUmber :', 451)
1.7183859
('Iteration NUmber :', 452)
2.0316176
('Iteratio

1.7879078
('Iteration NUmber :', 644)
1.6115437
('Iteration NUmber :', 645)
1.5132983
('Iteration NUmber :', 646)
1.3857073
('Iteration NUmber :', 647)
1.8641593
('Iteration NUmber :', 648)
1.7878535
('Iteration NUmber :', 649)
1.7160515
('Iteration NUmber :', 650)
1.5892084
('Iteration NUmber :', 651)
2.5256884
('Iteration NUmber :', 652)
1.3790352
('Iteration NUmber :', 653)
2.01925
('Iteration NUmber :', 654)
1.6124036
('Iteration NUmber :', 655)
1.7047155
('Iteration NUmber :', 656)
1.3529799
('Iteration NUmber :', 657)
1.9655906
('Iteration NUmber :', 658)
1.3433032
('Iteration NUmber :', 659)
2.2291803
('Iteration NUmber :', 660)
1.5410271
('Iteration NUmber :', 661)
1.4759104
('Iteration NUmber :', 662)
1.4680965
('Iteration NUmber :', 663)
1.4150277
('Iteration NUmber :', 664)
1.1148069
('Iteration NUmber :', 665)
1.3298333
('Iteration NUmber :', 666)
1.8001288
('Iteration NUmber :', 667)
1.3417077
('Iteration NUmber :', 668)
1.4818667
('Iteration NUmber :', 669)
1.189783
('Ite

1.3261085
('Iteration NUmber :', 861)
1.4407755
('Iteration NUmber :', 862)
1.0775359
('Iteration NUmber :', 863)
1.2023113
('Iteration NUmber :', 864)
1.9588814
('Iteration NUmber :', 865)
1.5847234
('Iteration NUmber :', 866)
1.0697056
('Iteration NUmber :', 867)
1.3558804
('Iteration NUmber :', 868)
1.3465595
('Iteration NUmber :', 869)
1.4362094
('Iteration NUmber :', 870)
0.8761072
('Iteration NUmber :', 871)
1.1374788
('Iteration NUmber :', 872)
1.4725211
('Iteration NUmber :', 873)
1.4237993
('Iteration NUmber :', 874)
1.816081
('Iteration NUmber :', 875)
1.399971
('Iteration NUmber :', 876)
1.4763314
('Iteration NUmber :', 877)
1.8119314
('Iteration NUmber :', 878)
2.1919723
('Iteration NUmber :', 879)
1.2022028
('Iteration NUmber :', 880)
1.5796409
('Iteration NUmber :', 881)
1.6231164
('Iteration NUmber :', 882)
1.232235
('Iteration NUmber :', 883)
1.8664843
('Iteration NUmber :', 884)
1.4580785
('Iteration NUmber :', 885)
1.5285621
('Iteration NUmber :', 886)
1.9592851
('Ite

...

0.39660698
('Iteration NUmber :', 1911)
0.49789065
('Iteration NUmber :', 1912)
0.58392584
('Iteration NUmber :', 1913)
0.6245185
('Iteration NUmber :', 1914)
0.67732584
('Iteration NUmber :', 1915)
0.82270736
('Iteration NUmber :', 1916)
0.570204
('Iteration NUmber :', 1917)
0.5564934
('Iteration NUmber :', 1918)
0.44279158
('Iteration NUmber :', 1919)
0.43476838
('Iteration NUmber :', 1920)
0.54839134
('Iteration NUmber :', 1921)
0.5815991
('Iteration NUmber :', 1922)
0.5283456
('Iteration NUmber :', 1923)
0.6878576
('Iteration NUmber :', 1924)
0.3834209
('Iteration NUmber :', 1925)
0.682655
('Iteration NUmber :', 1926)
0.7886041
('Iteration NUmber :', 1927)
0.6748775
('Iteration NUmber :', 1928)
0.4700232
('Iteration NUmber :', 1929)
0.44999894
('Iteration NUmber :', 1930)
0.4784958
('Iteration NUmber :', 1931)
0.6472403
('Iteration NUmber :', 1932)
0.68881315
('Iteration NUmber :', 1933)
0.6304841
('Iteration NUmber :', 1934)
0.43641976
('Iteration NUmber :', 1935)
0.4871723
('Iter

0.4774052
('Iteration NUmber :', 2119)
0.7145859
('Iteration NUmber :', 2120)
0.6575725
('Iteration NUmber :', 2121)
0.50394464
('Iteration NUmber :', 2122)
0.44393453
('Iteration NUmber :', 2123)
0.4778087
('Iteration NUmber :', 2124)
0.4569467
('Iteration NUmber :', 2125)
0.36855125
('Iteration NUmber :', 2126)
0.5219753
('Iteration NUmber :', 2127)
0.43139604
('Iteration NUmber :', 2128)
0.5102899
('Iteration NUmber :', 2129)
0.46772957
('Iteration NUmber :', 2130)
0.46387434
('Iteration NUmber :', 2131)
0.47882563
('Iteration NUmber :', 2132)
0.6185171
('Iteration NUmber :', 2133)
0.38202846
('Iteration NUmber :', 2134)
0.80862767
('Iteration NUmber :', 2135)
0.49452317
('Iteration NUmber :', 2136)
0.62605906
('Iteration NUmber :', 2137)
0.61303574
('Iteration NUmber :', 2138)
0.5107941
('Iteration NUmber :', 2139)
0.43008316
('Iteration NUmber :', 2140)
0.54976296
('Iteration NUmber :', 2141)
0.55863214
('Iteration NUmber :', 2142)
0.43160072
('Iteration NUmber :', 2143)
0.3778345

0.36531585
('Iteration NUmber :', 2327)
0.3980937
('Iteration NUmber :', 2328)
0.35290888
('Iteration NUmber :', 2329)
0.41377556
('Iteration NUmber :', 2330)
0.31254664
('Iteration NUmber :', 2331)
0.6329349
('Iteration NUmber :', 2332)
0.3898831
('Iteration NUmber :', 2333)
0.44921926
('Iteration NUmber :', 2334)
0.33229658
('Iteration NUmber :', 2335)
0.38503826
('Iteration NUmber :', 2336)
0.5786958
('Iteration NUmber :', 2337)
0.764412
('Iteration NUmber :', 2338)
0.57746226
('Iteration NUmber :', 2339)
0.3928697
('Iteration NUmber :', 2340)
0.36246833
('Iteration NUmber :', 2341)
0.33938232
('Iteration NUmber :', 2342)
0.52322036
('Iteration NUmber :', 2343)
0.48541695
('Iteration NUmber :', 2344)
0.48620707
('Iteration NUmber :', 2345)
0.51371205
('Iteration NUmber :', 2346)
0.3798526
('Iteration NUmber :', 2347)
0.32628828
('Iteration NUmber :', 2348)
0.38380992
('Iteration NUmber :', 2349)
0.4969644
('Iteration NUmber :', 2350)
0.40387696
('Iteration NUmber :', 2351)
0.5762592

0.39772767
('Iteration NUmber :', 2534)
0.31733158
('Iteration NUmber :', 2535)
0.66509193
('Iteration NUmber :', 2536)
0.34111542
('Iteration NUmber :', 2537)
0.34461966
('Iteration NUmber :', 2538)
0.39080292
('Iteration NUmber :', 2539)
0.3480297
('Iteration NUmber :', 2540)
0.36561006
('Iteration NUmber :', 2541)
0.36475194
('Iteration NUmber :', 2542)
0.523666
('Iteration NUmber :', 2543)
0.4669745
('Iteration NUmber :', 2544)
0.3185849
('Iteration NUmber :', 2545)
0.3458592
('Iteration NUmber :', 2546)
0.33441457
('Iteration NUmber :', 2547)
0.54786205
('Iteration NUmber :', 2548)
0.32184422
('Iteration NUmber :', 2549)
0.48102275
('Iteration NUmber :', 2550)
0.36857492
('Iteration NUmber :', 2551)
0.32629898
('Iteration NUmber :', 2552)
0.3452333
('Iteration NUmber :', 2553)
0.3552838
('Iteration NUmber :', 2554)
0.43142265
('Iteration NUmber :', 2555)
0.31414238
('Iteration NUmber :', 2556)
0.50806785
('Iteration NUmber :', 2557)
0.4233298
('Iteration NUmber :', 2558)
0.3653099

0.33666426
('Iteration NUmber :', 2741)
0.3669155
('Iteration NUmber :', 2742)
0.35751334
('Iteration NUmber :', 2743)
0.36088812
('Iteration NUmber :', 2744)
0.32801634
('Iteration NUmber :', 2745)
0.40844402
('Iteration NUmber :', 2746)
0.42244232
('Iteration NUmber :', 2747)
0.31911433
('Iteration NUmber :', 2748)
0.30808198
('Iteration NUmber :', 2749)
0.36168242
('Iteration NUmber :', 2750)
0.3275525
('Iteration NUmber :', 2751)
0.30779755
('Iteration NUmber :', 2752)
0.36563498
('Iteration NUmber :', 2753)
0.33160922
('Iteration NUmber :', 2754)
0.3247495
('Iteration NUmber :', 2755)
0.31748852
('Iteration NUmber :', 2756)
0.34204918
('Iteration NUmber :', 2757)
0.3784905
('Iteration NUmber :', 2758)
0.3819466
('Iteration NUmber :', 2759)
0.32803312
('Iteration NUmber :', 2760)
0.3265171
('Iteration NUmber :', 2761)
0.32987478
('Iteration NUmber :', 2762)
0.32083502
('Iteration NUmber :', 2763)
0.32033715
('Iteration NUmber :', 2764)
0.38182694
('Iteration NUmber :', 2765)
0.3370

In [ ]:
import matplotlib.pyplot as plt

radius = [1.0, 2.0, 3.0, 4.0]
area = [3.14159, 12.56636, 28.27431, 50.26544]

plt.plot(radius, area)
plt.show()

In [ ]:
predict = tf.argmax(logits, axis=1)
predict
batchLoss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=batch_label)
loss = tf.reduce_mean(batchLoss)
t_vars = tf.trainable_variables()



#tf.initialize_all_variables()
checkpoint_dir='/home/dl/Documents/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check_n'
saver = tf.train.Saver()
tf.initialize_all_variables()
with tf.Session() as sess:
	tf.train.start_queue_runners()
#	sess.run(tf.local_variables_initializer())
#	tf.global_variables_initializer().run()

	coord = tf.train.Coordinator()
	threads = tf.train.start_queue_runners(coord=coord)    
	ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    
	if ckpt and ckpt.model_checkpoint_path:
		print(ckpt.model_checkpoint_path)
		if os.path.isabs(ckpt.model_checkpoint_path):
			saver.restore(sess, ckpt.model_checkpoint_path)
		else:
			print("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzgdgdgd")
			saver.restore(sess, os.path.join(checkpoint_dir,ckpt.model_checkpoint_path))
		global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
		print('Successfully loaded model from %s at step=%s.' %(ckpt.model_checkpoint_path, global_step))



	for iteration in xrange(100000):
		print("Iteration NUmber :", iteration)        
		_trainY, _predict = sess.run([batch_label, predict])
        
		        

		_label = np.argmax(_trainY, axis=1)
		_accuracy = np.mean(_label == _predict)        
		print(_accuracy)
		coord.request_stop()
		coord.join(threads)
        

            